In [1]:
%pip install h2o

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 177.0 MB 131.3 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 91.5 MB/s eta 0:00:01
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=0533030546660b1c148809ab4577dcff647b0b881df5f2d4b3bf02c973cf12d2
  Stored in directory: /home/mk7516/.cache/pip/wheels/bf/d7/e0/905f059465fafaf07c9e79ba4c8d642ac646b01ea8c38b07d5
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=4f831c65e1c94dc6bcb6bd66006a00f5a1ba67ae0b30925cbe353bfd64aac98a
  Stored in directory: /home/mk7516/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built h2o future
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312-debug"; OpenJDK Runtime Environment (build 1.8.0_312-debug-b07); OpenJDK 64-Bit Server VM (build 25.312-b07-debug, mixed mode)
  Starting server from /home/mk7516/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /state/partition1/job-19156485/tmpm67nwqaf
  JVM stdout: /state/partition1/job-19156485/tmpm67nwqaf/h2o_mk7516_started_from_python.out
  JVM stderr: /state/partition1/job-19156485/tmpm67nwqaf/h2o_mk7516_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,07 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,22 days
H2O_cluster_name:,H2O_from_python_mk7516_u150lq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.541 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [4]:
test, train = dataSetup('australian')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
train.shape

(517, 15)

In [6]:
x = train.columns
y = "class"
x.remove(y)

In [7]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [8]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_1_AutoML_1_20220506_142744_model_95


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,33.0,33.0,21740.0,8.0,12.0,10.818182,30.0,52.0,47.575756




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.022022418832576832
RMSE: 0.1483995243677581
LogLoss: 0.11082597111600354
Mean Per-Class Error: 0.01788291538877864
AUC: 0.9988874857177221
AUCPR: 0.9987567961287924
Gini: 0.9977749714354442

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4801280789764455: 


,,0,1,Error,Rate
0,0,273.0,3.0,0.0109,(3.0/276.0)
1,1,6.0,235.0,0.0249,(6.0/241.0)
2,Total,279.0,238.0,0.0174,(9.0/517.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.887771,0.019202,0.894231,0.903846,0.854369,0.893204,0.893204
1,auc,0.929229,0.010440,0.939259,0.935257,0.912644,0.932946,0.926038
2,err,0.112229,0.019202,0.105769,0.096154,0.145631,0.106796,0.106796
3,err_count,11.600000,1.949359,11.000000,10.000000,15.000000,11.000000,11.000000
4,f0point5,0.880160,0.044678,0.885827,0.933610,0.810277,0.876777,0.894309
5,f1,0.879185,0.021720,0.891089,0.900000,0.845361,0.870588,0.888889
6,f2,0.879356,0.012853,0.896414,0.868726,0.883621,0.864486,0.883534
7,lift_top_group,2.157300,0.178511,2.080000,1.962264,2.288889,2.395349,2.060000
8,logloss,0.339233,0.034391,0.310588,0.330498,0.398424,0.335133,0.321521
9,max_per_class_error,0.142249,0.030801,0.111111,0.150943,0.189655,0.139535,0.120000



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-05-06 15:25:08,52.004 sec,0.0,0.498853,0.690854,0.500000,0.466151,1.000000,0.533849
1,,2022-05-06 15:25:08,52.022 sec,5.0,0.364060,0.447664,0.980847,0.980752,2.145228,0.069632
2,,2022-05-06 15:25:08,52.042 sec,10.0,0.289620,0.324527,0.986086,0.985582,2.145228,0.059961
3,,2022-05-06 15:25:08,52.062 sec,15.0,0.242392,0.247383,0.990574,0.990459,2.145228,0.044487
4,,2022-05-06 15:25:08,52.081 sec,20.0,0.211633,0.197082,0.993114,0.992948,2.145228,0.032882
5,,2022-05-06 15:25:08,52.102 sec,25.0,0.186880,0.158626,0.995866,0.995568,2.145228,0.032882
6,,2022-05-06 15:25:08,52.121 sec,30.0,0.163603,0.127328,0.997700,0.997489,2.145228,0.023211
7,,2022-05-06 15:25:08,52.135 sec,33.0,0.148400,0.110826,0.998887,0.998757,2.145228,0.017408



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,A8,266.960815,1.000000,0.458614
1,A9,70.785393,0.265153,0.121603
2,A2,54.938286,0.205792,0.094379
3,A3,37.117981,0.139039,0.063765
4,A13,34.818306,0.130425,0.059815
5,A7,26.069323,0.097652,0.044785
6,A5,25.925480,0.097113,0.044538
7,A14,17.399439,0.065176,0.029891
8,A10,17.305044,0.064822,0.029728
9,A12,8.257612,0.030932,0.014186


In [9]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GBM_grid_1_AutoML_1_20220506_142744_model_95,0.933054,0.327768,0.935981,0.121001,0.310863,0.0966357
StackedEnsemble_BestOfFamily_8_AutoML_1_20220506_142744,0.932129,0.329441,0.92834,0.125616,0.30819,0.0949813
GBM_grid_1_AutoML_1_20220506_142744_model_110,0.929949,0.328548,0.921915,0.124068,0.310866,0.0966378
GBM_lr_annealing_selection_AutoML_1_20220506_142744_select_model,0.929588,0.324786,0.925239,0.115303,0.307042,0.0942751
GBM_grid_1_AutoML_1_20220506_142744_model_73,0.929431,0.325259,0.927841,0.114514,0.307959,0.094839
GBM_grid_1_AutoML_1_20220506_142744_model_60,0.929107,0.329955,0.922343,0.11837,0.311706,0.0971606
GBM_grid_1_AutoML_1_20220506_142744_model_47,0.92901,0.326316,0.918358,0.115303,0.306278,0.0938059
StackedEnsemble_BestOfFamily_7_AutoML_1_20220506_142744,0.928867,0.332451,0.917199,0.123046,0.309946,0.0960668
GBM_grid_1_AutoML_1_20220506_142744_model_20,0.92801,0.329704,0.913473,0.113755,0.309163,0.0955818
GBM_grid_1_AutoML_1_20220506_142744_model_130,0.927732,0.329513,0.912448,0.10829,0.306994,0.094245
